# Section 1: Load Model

In [19]:
# For dataset
import pandas as pd
import json
import os
import ast
import re
import numpy as np
from datasets import Dataset
import matplotlib.pyplot as plt


import torch
from time import time
 
# For wandb
import wandb
# Set seed
import pickle
set_seed(42)
import seaborn as sns
from copy import deepcopy
from tqdm import tqdm
PATH = "0_Thesis/4_Multi/"

In [2]:
if os.getcwd() == '/root':
    new_path = "/root/0_Thesis/0_final/"
    os.chdir(new_path)
else:
    os.chdir("..") 
print(os.getcwd())

/root/0_Thesis/0_final


In [6]:
model_id = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
# model_id ="unsloth/Qwen2.5-14B-Instruct-bnb-4bit"
# model_id = "unsloth/gemma-2-9b-it-bnb-4bit"

In [9]:
df = pd.read_csv("lgb/lgb_with_llm_probs_binary.csv")

In [10]:
df.shape

(63512, 17)

In [12]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Label 1

In [13]:
df['true_label']   = 0
df.loc[df['hate_label_id'] == 1, 'true_label'] = 1

In [ ]:
X = df[['unsloth/Qwen2.5-14B-Instruct-bnb-4bit_label_1', 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit_label_1', 'unsloth/mistral-7b-instruct-v0.3-bnb-4bit_label_1', 'unsloth/gemma-2-9b-it-bnb-4bit_label_1']]
y = df['true_label']  

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)


train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

params = {
    'objective': 'binary', 
    'metric': 'binary_logloss', 
    'boosting_type': 'gbdt', 
    'num_leaves': 34, 
    'learning_rate': 0.05,
    'feature_fraction': 0.9, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 5, 
    'verbose': -1, 
}

model = lgb.train(
    params,
    train_data,
    valid_sets=[val_data],
    # verbose_eval=10,
)

val_preds = model.predict(X_val, num_iteration=model.best_iteration)
val_preds_binary = np.round(val_preds) 

accuracy = accuracy_score(y_val, val_preds_binary)
print(accuracy)
# model.booster_.save_model("lgb_model.json", format="json")

0.7928211586901763


# Label 2

In [18]:
X = df[['unsloth/Qwen2.5-14B-Instruct-bnb-4bit_label_2', 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit_label_2', 'unsloth/mistral-7b-instruct-v0.3-bnb-4bit_label_2', 'unsloth/gemma-2-9b-it-bnb-4bit_label_2']]
y = df['hate_label_id']  
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

params = {
    'objective': 'binary', 
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 34,
    'learning_rate': 0.05, 
    'feature_fraction': 0.9, 
    'bagging_fraction': 0.8,
    'bagging_freq': 5,  
    'verbose': -1, 
}

model = lgb.train(
    params,
    train_data,
    valid_sets=[val_data],
    num_boost_round=1000,

)
val_preds = model.predict(X_val, num_iteration=model.best_iteration)

# roc_auc_score(y_val, val_preds)
# with open("lgb_model_1.pkl", "wb") as f:
#     pickle.dump(model, f)